Notebook elaborado para apresentar conceitos sobre a sismicidade do Brasil.
Criado para a disciplina Geofísica I da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Os dados utilizados nesta demonstração foram obtidos através do catálogo de eventos da [Rede Sismográfica Brasileira (RSBR)](http://rsbr.gov.br/pevjs/evlist.html#brasil)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
emerson.almeida@ufu.br<br>

<p>&nbsp;</p>
Última modificação: 11/05/2021

# Instruções iniciais

Neste notebook você irá observar uma série de eventos sísmicos registrados no Brasil desde 1903 até o dia 02/05/2021. Leia atentamente as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, não é obrigatório que você conheça a programação para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

<p>&nbsp;</p>

### Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os módulos e funções que precisaremos no decorrer da demonstração. A célula abaixo contém as funções utilizadas nos cálculos e as funções utilizadas para gerar as figuras. Execute-a uma vez antes de executar qualquer outra parte do código e **não altere nada no código apresentado na célula** para não criar problemas na execução do código.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

<p>&nbsp;</p>

**Importante!** Nem todas as células irão retornar algum tipo de mensagem ou figura após a execução. Isto é normal e não significa que a execução tenha falhado. Perceba o símbolo **In [   ]:** próximo ao canto superior esquerdo de cada célula. Ao iniciar a execução de uma delas você verá um asterisco ($*$) aparecer dentro dos colchetes. Você saberá que a execução ocorreu sem problemas quando o $*$ dentro dos colchetes for substituído por um número e nenhuma mensagem de erro aparecer abaixo da célula executada. 

In [ ]:
# imports e funções

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader


class catalogo():
    
    def __init__(self, fname):
        self.arquivo = fname
        self.todos = pd.read_csv(fname, sep=";", decimal=".", usecols=np.arange(9))

        self.magnitudes = {'0.00 a 1.99': self.todos.loc[(self.todos['magnitude'] >=0.00) & (self.todos['magnitude'] < 1.99)],
                           '2.00 a 2.99': self.todos.loc[(self.todos['magnitude'] >=2.00) & (self.todos['magnitude'] < 2.99)],
                           '3.00 a 3.99': self.todos.loc[(self.todos['magnitude'] >=3.00) & (self.todos['magnitude'] < 3.99)],
                           '4.00 a 4.99': self.todos.loc[(self.todos['magnitude'] >=4.00) & (self.todos['magnitude'] < 4.99)],
                           '5.00 a 5.99': self.todos.loc[(self.todos['magnitude'] >=5.00) & (self.todos['magnitude'] < 5.99)],
                           '6.00 a 6.99': self.todos.loc[(self.todos['magnitude'] >=6.00) & (self.todos['magnitude'] < 6.99)],
                           '7.00+': self.todos.loc[(self.todos['magnitude'] >=7.00)]}
        
        self.hipocentros = {'0.0 a 1.0': self.todos.loc[(self.todos['depth'] <=1.0)],
                            '1.0 a 10.0': self.todos.loc[(self.todos['depth'] > 1.0) & (self.todos['depth'] <= 10.0)],
                            '10.0 a 40.0': self.todos.loc[(self.todos['depth'] > 10.0) & (self.todos['depth'] <= 40.0)],
                            '40.0 a 100.0': self.todos.loc[(self.todos['depth'] > 40.0) & (self.todos['depth'] <= 100.0)],
                            '100.0 a 700.0': self.todos.loc[(self.todos['depth'] > 100.0) & (self.todos['depth'] <= 700.0)]}

        
    def plota_epicentros(self, coords=None, dpi=96, res=[1280, 1024], extras=None):
        
        """
        coords = [lon_min, lon_max, lat_min, lat_max]
        
        area = [lon_min_box, lon_max_box, lat_min_box, lat_max_box]
        """

        paleta = ['black', 'blue', 'lightgreen', 'red', 'magenta', 'orange', 'yellow']
        contornos = ['none', 'black', 'black', 'black', 'black', 'black', 'black']
        legenda = [r'$M < 2.0$', r'$2.0 \leq M < 3.0$', r'$3.0 \leq M < 4.0$', r'$4.0 \leq M < 5.0$', \
                   r'$5.0 \leq M < 6.0$', r'$6.0 \leq M < 7.0$', r'$M \geq 7.0$']
        
        fig = plt.figure(figsize=(res[0]/dpi, res[1]/dpi), dpi=dpi/1.3)
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.stock_img()
        
        for j, k in enumerate(self.magnitudes.keys()):
            curr_m = self.magnitudes[k]
            ax.scatter(curr_m['longitude'], curr_m['latitude'], edgecolors=contornos[j], s=2**(j+3), \
                       c=paleta[j], label=legenda[j])
     
        leg = ax.legend(loc='lower left', ncol=2, fontsize='large', shadow=True, borderpad=0.75, \
                        title=r'MAGNITUDE ($M$)', title_fontsize=12, labelspacing=1)
        leg._legend_box.align = "left"

        if coords:
            lon_min, lon_max, lat_min, lat_max = coords
            ax.set_xlim(lon_min, lon_max)
            ax.set_ylim(lat_min, lat_max)
        
        if extras:
            for k in extras.keys():
                ax.scatter(extras[k][1], extras[k][0], marker='s', s=2**7, c='yellow', edgecolors='k')
                ax.text(extras[k][1], extras[k][0], k, horizontalalignment='right', \
                        verticalalignment='bottom', fontsize=16)
            
        ax.coastlines(resolution='50m', alpha=0.20)
        ax.add_feature(feature.BORDERS, linestyle='-', alpha=0.20)
        shape_feature = ShapelyFeature(Reader('UFEBRASIL.shp').geometries(), ccrs.PlateCarree(), facecolor='none', \
                                       linewidth=1, edgecolor = (0.5, 0.5, 0.5, 1))
        ax.add_feature(shape_feature, linestyle='-')
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
        ax.set_title('Epicentros (1903 - 2021)', fontsize=14)

        plt.show()
        
        
        
    def plota_hipocentros(self, coords=None, dpi=96, res=[1280, 1024], extras=None):
        
        paleta = ['blue', 'lightgreen', 'red', 'magenta', 'yellow']
        contornos = ['none', 'black', 'black', 'black', 'black']
        legenda = [r'$z < 1.0$ km', r'$1.0 \leq z < 10.0$ km', r'$10.0 \leq z < 40.0$ km', \
                   r'$40.0 \leq z < 100.0$ km', r'$100.0 \leq z < 700.0$ km']
        
        fig = plt.figure(figsize=(res[0]/dpi, res[1]/dpi), dpi=dpi/1.3)
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.stock_img()
        
        for j, k in enumerate(self.hipocentros.keys()):
            curr_h = self.hipocentros[k]
            ax.scatter(curr_h['longitude'], curr_h['latitude'], edgecolors=contornos[j], s=2**(j+4), \
                       c=paleta[j], label=legenda[j])
        
        leg = ax.legend(loc='lower left', ncol=2, fontsize='large', shadow=True, borderpad=0.75, \
                        title=r'PROFUNDIDADE ($z$)', title_fontsize=12, labelspacing=1)
        leg._legend_box.align = "left"
        
        if coords:
            lon_min, lon_max, lat_min, lat_max = coords
            ax.set_xlim(lon_min, lon_max)
            ax.set_ylim(lat_min, lat_max)
        
        if extras:
            for k in extras.keys():
                ax.scatter(extras[k][1], extras[k][0], marker='s', s=2**6.5, c='yellow', edgecolors='k')
                ax.text(extras[k][1], extras[k][0], k, horizontalalignment='right', \
                        verticalalignment='bottom', fontsize=16)
        
        ax.coastlines(resolution='50m', alpha=0.20)
        ax.add_feature(feature.BORDERS, linestyle='-', alpha=0.20)
        shape_feature = ShapelyFeature(Reader('UFEBRASIL.shp').geometries(), ccrs.PlateCarree(), facecolor='none', \
                                       linewidth=1, edgecolor = (0.5, 0.5, 0.5, 1))
        ax.add_feature(shape_feature, linestyle='-')
        ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
        ax.set_title('Hipocentros (1903 - 2021)', fontsize=14)

        plt.show()
        
        
    def plota_ocorrencia(self, dpi=96, res=[1280, 1024]):
        
        anos = [int(i.split('-')[0]) for i in self.todos['origin']]
        
        fig = plt.figure(figsize=(res[0]/dpi/2, res[1]/dpi/2), dpi=dpi/1.3)
        ax = plt.axes()
        ax.hist(anos, bins=1+(np.max(anos)-np.min(anos)), align='mid')
        ax.set_xlabel('Ano', fontsize=14)
        ax.set_ylabel('Nº de eventos registrados', fontsize=14)
        ax.set_title('Ocorrência de terremotos no Brasil \n(de acordo com o catálogo da RSBR)', fontsize=14)
        ax.grid()
        plt.show()


### Passo 02 - Carregar os dados

Esta demonstração utilizará dados carregados a partir de um arquivo ASCII em formato .csv contendo uma tabela com os eventos registrados no catálogo da Rede Sismográfica Brasileira entre 1903 e o início de maio de 2021. Execute a célula abaixo para carregar este arquivo.

In [ ]:
%matplotlib notebook
rsbr = catalogo('rsbr.csv')

### Passo 03 - Mapa de Epicentros

Comumente se considera o Brasil como um local onde não há ocorrência de terremotos. De fato, eventos catastróficos como os que ocorrem em países como Chile e Japão não ocorrem em nosso território, mas isso não significa que terremotos não ocorram por aqui.

Se você executar a célula abaixo você verá um histograma que mostra o número de eventos sísmicos registrado no Brasil ao longo o último século.

In [ ]:
rsbr.plota_ocorrencia()

O aumento do número de ocorrências na última década deve-se à instalação de novas estações sismográficas em território nacional, que puderam fornecer mais dados aos sismólogos e permitiram conhecer melhor a frequência com que estes eventos ocorrem em território nacional. Portanto, **isto não significa que a atividade sísmica no Brasil esteja aumentando**, e sim que os centros de pesquisa estão dispondo de mais recursos para registrar e a ocorrência dos tremores.

Agora execute a célula abaixo para gerar um mapa de epicentros destes eventos e visualizar onde eles ocorreram.

In [ ]:
regiao = [-80.0, -30.0, -46.0, 6.0]

rsbr.plota_epicentros(coords=regiao)

Você consegue perceber algum padrão na distribuição dos eventos sísmicos de acordo com suas magnitudes?

### Passo 04 - Mapa de Hipocentros

Os terremotos que ocorrem no Brasil costumam ocorrer em profundidades bastante limitadas. Execute a célula abaixo para gerar um mapa de hipocentros dos eventos apresentados no mapa anterior.

In [ ]:
rsbr.plota_hipocentros(coords=regiao)

Você percebe algum padrão em relação à profundidade em que os eventos ocorreram, comparando esta informação com o mapa de epicentros?

### Passo 05 - Visualizando regiões específicas

Agora volte à segunda célula do **Passo 03**, onde foram definidos os valores mínimos e máximos de longitude e de latitude e insira outras coordenadas para visualizar áreas do seu interesse. Você deve inserir os valores de longitude mínima, longitude máxima, latitutde mínima e latitude máxima, nesta ordem, entre os colchetes. Utilize um ponto (.) como separador decimal e uma vírgula para separar os valores de coordenadas.